In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
cd '/content/drive/My Drive/detection'


/content/drive/My Drive/detection


In [ ]:
train = pd.read_csv('train.csv',encoding="cp949")
test = pd.read_csv('test.csv',encoding="UTF8")

In [ ]:
print('train :',len(train)) # 훈련용 리뷰 개수 출력
print('tesy :',len(test)) # 테스트용 리뷰 개수 출력

train : 105431
tesy : 4896


In [ ]:
train[:6]

,id,문장,label
0,1,아내가 드디어 출산하게 되어서 정말 신이 나.,1
1,2,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,1
2,3,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,1
3,4,남편이 죽기 전에 손자들 얼굴 보고 싶다고 했는데 다행히 늦지 않아서 안도했어.,1
4,5,친구가 사준 로또가 당첨이 됐어.,1
5,6,시험을 잘 보지 못해서 부모님께 혼날까 봐 걱정했는데 오히려 다독여 주셔서 다행이야.,1


In [ ]:
train['문장'].nunique(), train['label'].nunique()

(100765, 2)

In [ ]:
train.drop_duplicates(subset=['문장'], inplace=True)

In [ ]:
print(test.isnull().values.any())

False


In [ ]:
train = train.dropna(how = 'any')

In [ ]:
train['문장'] = train['문장'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train[:6]

,id,문장,label
0,1,아내가 드디어 출산하게 되어서 정말 신이 나,1
1,2,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야 너무 행복해,1
2,3,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어,1
3,4,남편이 죽기 전에 손자들 얼굴 보고 싶다고 했는데 다행히 늦지 않아서 안도했어,1
4,5,친구가 사준 로또가 당첨이 됐어,1
5,6,시험을 잘 보지 못해서 부모님께 혼날까 봐 걱정했는데 오히려 다독여 주셔서 다행이야,1


In [ ]:
train['문장'].replace('', np.nan, inplace=True)
train = train.dropna(how = 'any')
print('전처리 후 훈련용 샘플의 개수 :',len(train))

전처리 후 훈련용 샘플의 개수 : 100765


In [ ]:
test.drop_duplicates(subset = ['문장'], inplace=True) # 문장 열에서 중복인 내용이 있다면 중복 제거
test['문장'] = test['문장'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test['문장'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test = test.dropna(how='any') # Null 값 제거
print('처리후 샘플수 :',len(test))

처리후 샘플수 : 4873


In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 7.1 MB/s 
     |████████████████████████████████| 86 kB 6.7 MB/s 
     |████████████████████████████████| 448 kB 75.4 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from konlpy.tag import Okt

In [ ]:
okt = Okt()
print(train['문장'][1])
okt.morphs(train['문장'][1], stem = True)

재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야 너무 행복해


['재',
 '취업',
 '이',
 '돼다',
 '받다',
 '되다',
 '첫',
 '월급',
 '으로',
 '온',
 '가족',
 '이',
 '외식',
 '을',
 '하다',
 '예정',
 '이야',
 '너무',
 '행복하다']

In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

xtrain = []
for sentence in train['문장']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거a
    xtrain.append(temp_X)

xtest = []
for sentence in test['문장']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    xtest.append(temp_X)

In [ ]:
print(xtrain[:3])

[['아내', '드디어', '출산', '되어다', '정말', '신', '나'], ['재', '취업', '돼다', '받다', '되다', '첫', '월급', '온', '가족', '외식', '을', '예정', '이야', '너무', '행복하다'], ['빚', '을', '드디어', '다', '갚다', '되어다', '이제야', '안도', '감', '들다']]


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(xtrain)
print(tokenizer.word_index)
print(len(tokenizer.word_index))
with open("vocab_dict.json", "w") as json_file:
    json.dump(tokenizer.word_index, json_file)

{'을': 1, '내': 2, '있다': 3, '나': 4, '것': 5, '같다': 6, '너무': 7, '친구': 8, '보다': 9, '없다': 10, '좋다': 11, '않다': 12, '안': 13, '되다': 14, '말': 15, '에게': 16, '못': 17, '생각': 18, '싶다': 19, '에서': 20, '사람': 21, '받다': 22, '이야': 23, '들다': 24, '더': 25, '만': 26, '게': 27, '수': 28, '모르다': 29, '해': 30, '자다': 31, '그렇다': 32, '하고': 33, '정말': 34, '일': 35, '돈': 36, '어떻다': 37, '한테': 38, '나다': 39, '돼다': 40, '가다': 41, '고': 42, '거': 43, '마음': 44, '많이': 45, '걱정': 46, '로': 47, '다': 48, '기분': 49, '때': 50, '아내': 51, '그': 52, '해보다': 53, '아니다': 54, '힘들다': 55, '남편': 56, '나르다': 57, '응': 58, '회사': 59, '요즘': 60, '오늘': 61, '적': 62, '먹다': 63, '거야': 64, '때문': 65, '스럽다': 66, '결혼': 67, '기쁘다': 68, '많다': 69, '지금': 70, '야': 71, '오다': 72, '이제': 73, '우리': 74, '이번': 75, '다른': 76, '집': 77, '준비': 78, '가족': 79, '주다': 80, '아프다': 81, '아들': 82, '일이': 83, '해주다': 84, '얘기': 85, '맞다': 86, '화가': 87, '건강': 88, '자식': 89, '부모님': 90, '나오다': 91, '라고': 92, '슬프다': 93, '엄마': 94, '왜': 95, '나이': 96, '되어다': 97, '직장': 98, '병원': 99, '걸': 100, '열심히': 101, '이야기':

In [ ]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트

for key, value in tokenizer.word_counts.items():
    if(value < threshold):
        rare_cnt = rare_cnt + 1

print('전체단어 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))

전체단어 : 13012
등장 빈도가 2번 이하인 희귀 단어의 수: 5188


In [ ]:
vocab_size = total_cnt - rare_cnt + 2
print('공통 단어 종류 :',vocab_size)

공통 단어 종류 : 7826


In [ ]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(xtrain)
xtrain = tokenizer.texts_to_sequences(xtrain)
xtest = tokenizer.texts_to_sequences(xtest)

In [ ]:
max_len = 30
xtrain = pad_sequences(xtrain, maxlen = max_len)
xtest = pad_sequences(xtest, maxlen = max_len)

In [ ]:
y_train = np.array(train['label'])
y_test = np.array(test['label'])
xtrain[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 1731,  123,   41,   23,   15, 1014,  643,
        526,   80, 1316,    2, 1207,   24,    8,  257], dtype=int32)

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM,SimpleRNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         782600    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               117248    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 899,977
Trainable params: 899,977
Non-trainable params: 0
_________________________________________________________________


In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('mymodel.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(xtrain, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

Epoch 1/15
1344/1344 [==============================] - 95s 67ms/step - loss: 0.5098 - acc: 0.7490 - val_loss: 0.5043 - val_acc: 0.8167

Epoch 00001: val_acc improved from -inf to 0.81665, saving model to mymodel.h5
Epoch 2/15
1344/1344 [==============================] - 90s 67ms/step - loss: 0.4528 - acc: 0.7849 - val_loss: 0.4656 - val_acc: 0.8316

Epoch 00002: val_acc improved from 0.81665 to 0.83159, saving model to mymodel.h5
Epoch 3/15
1344/1344 [==============================] - 91s 68ms/step - loss: 0.4142 - acc: 0.8073 - val_loss: 0.4742 - val_acc: 0.8227

Epoch 00003: val_acc did not improve from 0.83159
Epoch 4/15
1344/1344 [==============================] - 92s 68ms/step - loss: 0.3788 - acc: 0.8265 - val_loss: 0.6055 - val_acc: 0.7360

Epoch 00004: val_acc did not improve from 0.83159
Epoch 5/15
1344/1344 [==============================] - 87s 65ms/step - loss: 0.3468 - acc: 0.8426 - val_loss: 0.5073 - val_acc: 0.8126

Epoch 00005: val_acc did not improve from 0.83159
Epoc

In [ ]:
loaded_model = load_model('mymodel.h5')
model.save('mymodel.h5')
print(loaded_model.predict(xtest[:5]))
print(y_test[:5])

[[0.9139227 ]
 [0.96168256]
 [0.85899043]
 [0.4899372 ]
 [0.9873872 ]]
[1 1 1 1 1]


In [ ]:
from tensorflow import keras
model = keras.models.load_model('mymodel.h5', compile=True)
 
export_path = 'dentection'
model.save(export_path, save_format="tf")


INFO:tensorflow:Assets written to: dentection/assets


INFO:tensorflow:Assets written to: dentection/assets


In [ ]:
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('detection') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('mymodel.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(xtest, y_test)[1]))

153/153 [==============================] - 2s 9ms/step - loss: 0.4483 - acc: 0.7966

 테스트 정확도: 0.7966


In [ ]:
def s_analysis(new_sentence):
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 게임가능.\n".format(score * 100))
  else:
    print("{:.2f}% 게임불가 \n".format((1 - score) * 100))

In [ ]:
s_analysis("야구장가기")

71.16% 게임가능.



In [ ]:
s_analysis("즐거운 축구관람")

99.45% 게임가능.



In [ ]:
s_analysis("싫은 사장님과 회의")

79.06% 게임불가 



In [ ]:
s_analysis("싸운 아내와 가족회의")

85.31% 게임불가 

